In [1]:
# Importa o pacote principal do PyTorch para trabalhar com tensores e operações em CPU/GPU.
import torch

# Do PyTorch, importa o submódulo 'nn', que contém camadas, perdas e utilitários de redes neurais.
from torch import nn

# Importa do scikit-image o submódulo 'data', que fornece imagens de exemplo para testes/demonstrações.
from skimage import data


## Pooling

Documentação: https://pytorch.org/docs/stable/nn.html#torch.nn.MaxPool2d

```python
torch.nn.MaxPool2d(kernel_size, stride=None, padding=0)
```

**```kernel_size```** <br>
Tamanho dos *Field of View*. Pode ser uma tupla ou um único número. Ex: ```kernel_size = 3``` definirá FoV de $3 \times 3$

**```stride```** <br>
Controla o pulo da janela deslizante. 

**```padding```** <br>
Preenchimento com zeros nas bordas da imagem.

A camda de pooling espera uma entrada de **pelo menos** 3 dimensões ($C \times H \times W$), mas em geral a rede irá prover também a dimensão do batch ($B \times C \times H \times W$) 

In [2]:
# Cria um tensor 3D (C×H×W) com 1 canal e imagem 3×3; serve como exemplo simples para pooling 2D.
tns = torch.FloatTensor([ [ [ 1, 2,3 ], 
                            [4,5,6], 
                            [7,8,9]  ] ] )

# Define uma camada de MaxPooling 2D com janela 2×2 e stride=1 (anda 1 pixel por vez).
pool = nn.MaxPool2d(2, stride=1)

# Aplica o max-pool ao tensor; resultado terá dimensões (1, 2, 2) com os máximos de cada janela 2×2.
saida = pool(tns)

# Mostra as dimensões do tensor de entrada (esperado: torch.Size([1, 3, 3])).
print(tns.size())

# Imprime o conteúdo da entrada para conferência.
print(tns)

# Mostra as dimensões da saída (esperado: torch.Size([1, 2, 2])).
print(saida.size())

# Imprime o conteúdo da saída; deve ser [[5, 6], [8, 9]].
print(saida)


torch.Size([1, 3, 3])
tensor([[[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]])
torch.Size([1, 2, 2])
tensor([[[5., 6.],
         [8., 9.]]])


Ao processar dados com múltiplos canais, a camada de pooling processa cada canal de entrada separadamente ao invés de processar todos os canais como em uma camada convolucional. Isso significa que **o número de canais de saída para a camada de pooling é o mesmo que o número de canais de entrada**. 

Vamos processar abaixo a imagem da astronauta.

In [3]:
# Cria uma camada convolucional 2D para imagens RGB (3 canais) com 16 filtros 3×3;
# padding=1 preserva H e W (stride padrão = 1). Pesos iniciam aleatoriamente.
conv = nn.Conv2d(in_channels=3, out_channels=16, 
                 kernel_size=3, padding=1)

# Carrega a imagem de exemplo "astronaut" (NumPy, formato H×W×C) do skimage.
rgb = data.astronaut()

# Converte o array NumPy para tensor float32 do PyTorch (valores ainda em 0–255).
rgb_tns = torch.Tensor(rgb)

# Reorganiza para CHW e adiciona a dimensão de batch (N=1): (1, 3, H, W).
rgb_tns = rgb_tns.permute(2, 0, 1).unsqueeze(0)

# Aplica a convolução, gerando 16 mapas de ativação do mesmo tamanho espacial da entrada.
mapa_de_ativacao = conv(rgb_tns)

# Imprime o shape do feature map: esperado (1, 16, H, W).
print('Feature Map:', mapa_de_ativacao.shape)


Feature Map: torch.Size([1, 16, 512, 512])


In [4]:
# Cria uma camada de max-pooling 2×2; como não foi passado stride, ele assume stride=kernel_size (=2), reduzindo H e W pela metade.
pool = nn.MaxPool2d(kernel_size=2)

# Aplica o pooling aos mapas de ativação (formato típico: N×C×H×W) para obter mapas mais compactos.
saida = pool(mapa_de_ativacao)

# Exibe o novo tamanho (esperado: N×C×H/2×W/2 quando H e W são pares).
print(saida.size())


torch.Size([1, 16, 256, 256])


## Batch Normalization

Documentação: https://pytorch.org/docs/stable/nn.html#torch.nn.BatchNorm2d

```python
torch.nn.BatchNorm2d(num_features)
```

**```num_features```**<br>
$\mathbf{\gamma}$ e $\mathbf{\beta}$ são aprendidos individualmente para cada canal da entrada. Em ativações de camadas intermediárias, esse valor corresponde ao **número de feature maps**. 


In [5]:
# Define um bloco sequencial: Conv(3→32, k=3, p=1) + BatchNorm + ReLU + MaxPool(10×10).
blococonv = nn.Sequential(
    # Convolução 3×3 que preserva H×W (padding=1); entrada RGB (3 canais) → 32 mapas.
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    # Normalização por canal (32 canais) para estabilizar e acelerar o treino/inferência.
    nn.BatchNorm2d(32),
    # Não linearidade ReLU.
    nn.ReLU(),
    # Max-Pooling 10×10 (stride=10 por padrão): reduz H e W em ~10×.
    nn.MaxPool2d(kernel_size=10)
)
# Imprime a arquitetura do bloco para inspeção.
print(blococonv)

# Cria um minibatch empilhando 12 cópias de rgb_tns ao longo da dimensão do batch (dim=0).
minibatch = torch.cat(12 * [rgb_tns])

# Mostra o shape do minibatch (esperado: [12, 3, H, W]).
print(minibatch.size())
# Executa o forward: aplica conv→bn→relu→pool ao minibatch.
saida = blococonv(minibatch)
# Mostra o shape da saída (≈ [12, 32, H/10, W/10], arredondando para baixo se H/W não forem múltiplos de 10).
print(saida.size())


Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=10, stride=10, padding=0, dilation=1, ceil_mode=False)
)
torch.Size([12, 3, 512, 512])
torch.Size([12, 32, 51, 51])
